In [1]:
import io
import os
import re
import sys
import csv
import gzip
import time
import json
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
# import sqlite3
# from PIL import Image
import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
# from collections import Counter
# import seaborn as sns
# import matplotlib.colors as mcolors
# from matplotlib.ticker import FuncFormatter
import itertools
import collections
import pyreadr
import pywfa
from IPython.display import display_html
# import patchworklib as pw
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase import *
from hla_align import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

# Genuine logics changed from the main script were marked by string **X** in the original R files.
# Simplicity changes were marked by string **Y**

<Figure size 100x100 with 0 Axes>

In [2]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0482.bam"

# db = process_db_genfile(gene, ipd_gen_file_dir, hla_gene_information)
# db.to_csv(f'/well/band/users/rbx225/recyclable_files/hla_reference_files/v3570_aligners/{gene}.ssv', sep = ' ', index = False, header = True)

In [3]:
db = pd.read_csv(f'/well/band/users/rbx225/recyclable_files/hla_reference_files/v3570_aligners/{gene}.ssv', sep = ' ')

regstart = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Start'].values[0]
regend = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['End'].values[0]
strand = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Strand'].values[0]

reads_apart_max = 1000

command = f"samtools view {bam} chr6:{regstart-reads_apart_max}-{regend+reads_apart_max}"
reads = subprocess.run(command, shell = True, capture_output = True, text = True).stdout[:-1].split('\n')

reads = [i.split('\t') for i in reads if '##' not in i]
reads = pd.DataFrame(reads).iloc[:,:11]
reads.columns = [
    "ID", "flag", "chr", "pos", "map_quality", "CIGAR", "chr_alt", "pos_alt", "insert_size", "sequence", "base_quality"
]

reads['pos'] = reads['pos'].astype(int)
reads['pos_alt'] = reads['pos_alt'].astype(int)

reads = reads[
    (reads['chr'] == 'chr6') &
    (reads['chr_alt'].isin(['chr6', '='])) &
    (reads['pos_alt'] + reads['sequence'].str.len() >= np.ones(len(reads))*(regstart - reads_apart_max)) & 
    (reads['pos_alt'] <= np.ones(len(reads))*(regend + reads_apart_max))
]

id_counts = reads['ID'].value_counts()
valid_ids_ary = id_counts[id_counts == 2].index.tolist()
reads = reads[reads['ID'].isin(valid_ids_ary)].sort_values(by = 'ID').reset_index(drop = True)

reads['rev_seq'] = reads['sequence'].apply(reverse_complement)
reads['rev_bq'] = reads['base_quality'].apply(lambda bq: bq[::-1])

scores_mat = np.zeros((reads.shape[0], db.shape[1]))
rev_scores_mat = np.zeros((reads.shape[0], db.shape[1]))
for j, a in enumerate(db.columns[:5]):
    refseq = (''.join(db[a].tolist())).replace('.', '')
    ref = pywfa.WavefrontAligner(refseq)
    for i, (seq, bq) in enumerate(zip(reads['sequence'], reads['base_quality'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        cigars = result.cigartuples
        score = result.score
        cigars, score = adjust_align_score(cigars, score)
        likelihood_per_read_per_allele = calculate_score_per_alignment(seq, refseq_aligned, bq)
        scores_mat[i, j] = likelihood_per_read_per_allele
    for i, (seq, bq) in enumerate(zip(reads['rev_seq'], reads['rev_bq'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        cigars = result.cigartuples
        score = result.score
        cigars, score = adjust_align_score(cigars, score)
        likelihood_per_read_per_allele = calculate_score_per_alignment(seq, refseq_aligned, bq)
        rev_scores_mat[i, j] = likelihood_per_read_per_allele

scores_mat = np.maximum(scores_mat, rev_scores_mat)
        
temperature = 100
likelihood_mat = np.exp(scores_mat/temperature)/np.sum(np.exp(scores_mat/temperature), axis = 1, keepdims = True)
likelihood_mat